2. Create a runner notebook using papermill to run the template notebook across all tokens in the database and the last 2 complete weeks in the database.  Summarize the following in a single DataFrame in the runner notebook
    - avg hourly return by run
    - avg volatility by run
    - maximum drawdown over the entire period of each run
    - range of last close price - first close price

# Rachel Cao   UCB-Preprogram Course Python HW4 due on 1/3/22 

## Runner Notebook

Create the imports for the notebook

In [ ]:
import papermill as pm
import scrapbook as sb
import pandas as pd
from datetime import datetime, timedelta
import os

## The last two complete weeks in the database

This is Monday to Sunday, Monday is 0 and Sunday is 6

In [ ]:
def last_day_of_week(d: datetime, dow: int) -> datetime:
    #this is to find the last day of the week, [6]
    dow_d = d.weekday()
    days_delta = (7 - dow) + dow_d if dow_d < dow else dow_d - dow
    return d - timedelta(days=days_delta)

## Find the number of tokens in the database with SELECT distinct token FROM ohlc

In [ ]:
run_dates = [(last_day_of_week(datetime(2021, 11, 1), 6) + timedelta(days=14)*x) for x in range(1)]
tokens = ['BTC','ETH','USDT','SOL','ADA','DOT','AVAX','ATOM','CRV','AAVE','COMP']
connection_string = 'sqlite:///data/data.db'
#events_db = 'crypto_events_database'

The directory for the output folder

In [ ]:
notebook4_output_folder = os.path.expanduser('hw4_outputs')

if not os.path.exists(notebook4_output_folder):
    os.makedirs(notebook4_output_folder)

### Run the same notebook code for each separate token

The template notebook then becomes functional to output multiple notebooks, one for each token

In [ ]:
for run_date in run_dates:
    for token in tokens:
        print(f'running for {token} for date {run_date} to {run_date + timedelta(days=14)}', end = '\r')
        res = pm.execute_notebook(
            'template_hw_4_rachel_cao',
            f'{notebook4_output_folder}/template_hw_4_rachel_cao_{token}_{run_date}.ipynb',
            parameters = {
                'from_date' : f'{run_date}',
                'to_date' : f'{run_date + timedelta(days=14)}',
                'token' : token,
                'connection_string' : connection_string
                #'events_db' : events_db
            }
        )

Let's check one of the notebooks

In [ ]:
nb = sb.read_notebook('hw4_outputs/template_hw_4_rachel_cao_BTC_2021-11-05 00:00:00.ipynb')

In [ ]:
nb.scraps['prices'].data

## Iterate through each of the notebooks in the output folder to create a notebooks object

In [ ]:
nbs = sb.read_notebooks('hw4_outputs')

### Create a dataframe out of the notebooks

In [ ]:
pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'from_date': scrap['from_date'].data,
        'to_date': scrap['to_date'].data,
        'hourly_return': scrap['prices'].data.hourly_return.mean(),
        'volatility': scrap['prices'].data.volatility.mean(),
        'max_drawdown': scrap['prices'].data.max_drawdown.max(),
        'range_close':scrap['prices'].data.close[-1] - scrap['prices'].data.close[0]
    })
    for book, scrap in nbs.notebook_scraps.items()
])